# Retina Net 

En este documento se aplicará el modelo Retina Net, el cual es una mejora del algoritmo Faster RCNN. Este algoritmo descrito [aquí](https://arxiv.org/pdf/1708.02002.pdf) fué implementado usando el framework de DL Keras. Utilizaremos la implementación que se encuentra en [este repositorio](https://github.com/fizyr/keras-retinanet)

In [1]:
from keras_retinanet.models import load_model